In [1]:
import json
import uuid 
from elasticsearch import helpers
from elasticsearch import Elasticsearch

## read the josn file

In [3]:
with open("the json file path", 'r') as data_file:
    data = json.load(data_file)

### slice the data

In [4]:
data  = data['features']

In [5]:
data_elasticsearch = []

In [6]:
for i in data:
    temp = {}
    
    temp['name'] = i['properties']['name']
    temp["location"]={'lat':i['geometry']['coordinates'][1],
                     'lon':i['geometry']['coordinates'][0]}
    data_elasticsearch.append(temp)

### conect to the elasticsearch on the cloud

In [12]:
es = Elasticsearch("https://06d4ec43aa3e4460b5239afae619b838.us-central1.gcp.cloud.es.io:9243/"
                   ,http_auth=('name', 'pass'))

index_name = "places"

### create the indeax

In [13]:
if es.indices.exists(index_name):
    es.indices.delete(index=index_name)
    
settings = {
  "mappings": {
        "dynamic": "strict",
        "properties": {
          "name": {"type": "text"},
          "location": {"type": "geo_point"}}
        }
      }



    
es.indices.create(index=index_name, ignore=400, body=settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'places'}

### add meta data to data and bulk insertion to elasticsearch


In [14]:
actions = []
  
for json_row in data_elasticsearch:
    
 
        
        
    doc = { '_op_type': 'index',
            '_index': index_name,
            '_type': '_doc',
            '_id': uuid.uuid1(),
            '_source': json_row}

    actions.append(doc)
   
helpers.bulk(es, actions)
       
      


(44467, [])